Build (or rebuild) the ``pillars`` package. Run ``"make -C .. build" !`` **ONLY WHEN** the local package is out-of-date.

In [ ]:
import sys.process._
//"make -C .. build" !

Load the library dependencies (see ``load-ivy.sc`` for details).

In [ ]:
val ivy_path = System.getProperty("user.dir") + "/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(ivy_path)))

Define a block that performs computation between the selected input and an immediate operand.

In [ ]:
import tetriski.pillars.core.{BlockTrait, OpEnum}
import tetriski.pillars.core.OpEnum.OpEnum
import tetriski.pillars.hardware.PillarsConfig
import tetriski.pillars.archlib.{ElementAlu, ElementConst, ElementMux}

/** A subblock that performs computation between the selected input and a immediate operand.
 *
 * @constructor create an abstract block model that performs computation between the input and a immediate operand
 * @param name the name of the model
 */
class BlockImmediate(name: String) extends BlockTrait {
  val aluParams = List(32)
  val aluOpList = List(OpEnum.ADD)
  val muxParams = List(2, 32)
  val constParams = List(PillarsConfig.CONST_WIDTH)
  initName(name)

  addInPorts(Array("in0", "in1"))
  addOutPorts(Array("out0"))
  setConfigRegion()

  /** A multiplexer that can choose a data source for the port "inputA" of the ALU.
   */
  val mux0 = new ElementMux("mux0", muxParams)
  mux0.addInPorts(Array("input0", "input1"))
  mux0.addOutPorts(Array("out0"))
  addElement(mux0)

  /** An ALU that can perform some operations.
   */
  val alu0 = new ElementAlu("alu0", aluOpList, supBypass = true, aluParams)
  alu0.addInPorts(Array("inputA", "inputB"))
  alu0.addOutPorts(Array("out0"))
  addElement(alu0)

  /** A const unit connected to the port "inputB" of ALU.
   */
  val const0 = new ElementConst("const0", constParams)
  const0.addOutPorts(Array("out0"))
  addElement(const0)

  /** Interconnection inside this block.
   */
  addConnect(term("in0") -> mux0 / "input0")
  addConnect(term("in1") -> mux0 / "input1")
  addConnect(mux0 / "out0" -> alu0 / "inputA")
  addConnect(const0 / "out0" -> alu0 / "inputB")
  addConnect(alu0 / "out0" -> term("out0"))
}

Define an architecture and generate RTL code.

In [ ]:
import tetriski.pillars.core.{ArchitectureHierarchy, Connect, HardwareGenerator}
import tetriski.pillars.hardware.TopModule

val arch = new ArchitectureHierarchy()

val blockImm = new BlockImmediate("blockImm")
arch.addBlock(blockImm)

arch.addInPorts(Array("in0", "in1"))
arch.addOutPorts(Array("out0"))

arch.addConnect(arch.term("in0") -> blockImm/"in0")
arch.addConnect(arch.term("in1") -> blockImm/"in1")
arch.addConnect(blockImm/"out0" -> arch.term("out0"))

arch.init()

val connect = new Connect(arch.connectArray)
val hardwareGenerator = new HardwareGenerator(arch, connect)
val topDesign = () => new TopModule(
    hardwareGenerator.pillarsModuleInfo, hardwareGenerator.connectMap,
    hardwareGenerator.regionList, blockImm.aluParams(0)
)

chisel3.Driver.execute(Array("-td", "RTL/"), topDesign)